# Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MaxAbsScaler

plt.rcParams["figure.figsize"] = (32,16)
plt.rcParams['figure.dpi'] = 100

# Functions

In [ ]:
def weighted_mean_absolute_percentage_error(true, pred, mean=False):
    true = np.array(true)
    pred = np.array(pred)
    if true.ndim == 1:
        true = true.reshape(1,true.shape[0])
        pred = pred.reshape(1,pred.shape[0])
    wmape = np.abs(pred-true).sum(axis=1) / np.abs(true).sum(axis=1) * 100
    return wmape

In [ ]:
class Particle:
    def __init__(self, pos, bounds):
        self.pbest_fit = float('inf')
        self.bounds = bounds
        self.position = np.array(pos)
        self.velocity = np.array([0]*self.position.shape[0])
        self.pbest_pos = np.copy(self.position)
                
    def fly(self):
        for i in range(self.position.shape[0]):
            new_pos = np.copy(self.position[i] + self.velocity[i])
            if new_pos > self.bounds[1] or new_pos < self.bounds[0]:
                new_pos = np.random.uniform(self.bounds[0], self.bounds[1])
            self.position[i] = new_pos
            
class PSO:
    def __init__(self, n_particles, iters, bounds, dims, y_true, phases, W=0.7, C1=1, C2=1, 
                 C1_decay=1e-1, C2_decay=1e-1, mode='random', weights=[1,1,1,1]):
        self.W, self.C1, self.C2 = W, C1, C2
        self.C1_init, self.C2_init = C1, C2
        self.C1_decay, self.C2_decay = C1_decay, C2_decay
        self.iters = iters
        self.n_particles = n_particles
        self.particles = []
        self.y_true = y_true
        self.phases = phases
        self.gbest_hist = []
        self.weights = weights
        
        self.gbest_fit = float('inf')
        self.gbest_pos = np.array(self.init(bounds, dims))
        
        if mode == 'equal' or mode == 'equal_random':
            self.init_values_equal(bounds, dims)
        for i in range(n_particles):
            if mode == 'equal_random':
                r = np.random.uniform(0, 1)
                if r <= 0.5:
                    mode = 'random'
                else:
                    mode = 'equal'
            self.particles.append(Particle(self.init(bounds, dims, mode), bounds))
        
    def set_C1(self, iteration):
        self.C1 = self.C1_init * (1 / (1 + self.C1_decay * iteration))
        
    def set_C2(self, iteration):
        self.C2 = self.C2_init * (1 + self.C2_decay * iteration)
    
    def init_values_equal(self, bounds, dims):
        values = np.linspace(bounds[0], bounds[1], 8)
        comb = list(combinations_with_replacement(values, dims))
        r = np.array(random.sample(comb, self.n_particles))
        self.r = r
    
    def init(self, bounds, dims, mode='random', i=0):
        if mode == 'random':
            return [np.random.uniform(bounds[0], bounds[1]) for _ in range(dims)]
        elif mode == 'equal':
            return self.r[i]
            
                      
    def calculate_fitness(self):
        positions = []
        for p in self.particles:
            positions.append(list(np.copy(p.position)) + self.phases)
        positions = np.array(positions)
        
        y_preds = model.predict(positions, verbose=0, batch_size=2048)
        errors = np.array(tf.keras.metrics.mean_squared_error(self.y_true, y_preds))
        return errors
    
    def calculate_fitness2(self):
        positions = []
        for p in self.particles:
            pos = np.copy(p.position)
            for ph in self.phases:
                positions.append(list(pos)+ list([ph]))
        positions = np.array(positions)
        y_preds = model.predict(positions, verbose=0, batch_size=2048)
        y_preds = y_preds.reshape(self.y_true.shape[0], self.y_true.shape[1])
        errors = np.zeros(self.n_particles)
        for i in range(len(self.phases)):
            y_true_tmp, y_preds_tmp = self.y_true[:,i*128:(i+1)*128], y_preds[:,i*128:(i+1)*128]
            errors += tf.keras.metrics.mean_squared_error(y_true_tmp[:,:32], y_preds_tmp[:,:32])*self.weights[0] + tf.keras.metrics.mean_squared_error(y_true_tmp[:,32:64], y_preds_tmp[:,32:64])*self.weights[1] + tf.keras.metrics.mean_squared_error(y_true_tmp[:,64:96], y_preds_tmp[:,64:96])*self.weights[2] + tf.keras.metrics.mean_squared_error(y_true_tmp[:,96:], y_preds_tmp[:,96:])*self.weights[3]
            errors /= len(self.weights)
        errors /= len(self.phases)
        #print(y_preds.shape)
        
        errors = np.array(tf.keras.metrics.mean_squared_error(self.y_true, y_preds))
        return errors
    
    def set_pbest_gbest(self, iteration):
        fits = self.calculate_fitness2()
        i = 0
        for fit in fits:
            if fit < self.particles[i].pbest_fit:
                self.particles[i].pbest_fit = fit
                self.particles[i].pbest_pos = np.copy(self.particles[i].position)
            if fit < self.gbest_fit:
                self.gbest_fit = fit
                self.gbest_pos = np.copy(self.particles[i].position)
                self.gbest_ith = iteration
            i += 1
                     
    def fly(self, W ,C1, C2):
        for p in self.particles:
            velocity = (W*p.velocity) + (C1*np.random.random())*(p.pbest_pos - p.position) + (C2*np.random.random())*(self.gbest_pos - p.position)
            p.velocity = np.copy(velocity)
            p.fly()    
        
    def fit(self, tol=None, stuck=None, verbose=-1):
        i = 0
        stuck_flag = False
        stuck_counter = 0
        while i < self.iters and not stuck_flag: 
            gbest_prev = self.gbest_fit
            self.fly(W=self.W, C1=self.C1, C2=self.C2)
            gbest_prev = self.gbest_fit
            self.set_pbest_gbest(i)
            self.gbest_hist.append(self.gbest_fit)
            #self.set_C1(i)
            #self.set_C2(i)
            i += 1
            if stuck:
                if gbest_prev == self.gbest_fit:
                    stuck_counter += 1
                else:
                    stuck_counter = 0
                    
                if stuck_counter == stuck:
                    stuck_flag = True
                    
            if tol:
                if self.gbest_fit <= tol:
                    stuck_flag = True

In [ ]:
def make_result_csv(y_test, y_pred):
    e = ['I', 'Q', 'U', 'V']
    data = {'stokes I MSE': [], 'stokes Q MSE': [], 'stokes U MSE': [], 'stokes V MSE': [], 'stokes I WMAPE': [], 'stokes Q WMAPE': [], 'stokes U WMAPE': [], 'stokes V WMAPE': [], 'Amplitude stokes I':[], 'Amplitude stokes Q':[], 'Amplitude stokes U':[], 'Amplitude stokes V':[]}
    for i in range(4):
        wmape = weighted_mean_absolute_percentage_error(y_test[:,i*32:(i+1)*32], y_pred[:,i*32:(i+1)*32])
        mse = tf.keras.metrics.mean_squared_error(y_test[:,i*32:(i+1)*32], y_pred[:,i*32:(i+1)*32]).numpy()
        data['stokes ' + e[i] + ' MSE'] = mse
        data['stokes ' + e[i] + ' WMAPE'] = wmape
        if i != 0:
            data['Amplitude stokes ' + e[i]] = np.max(np.abs(y_test[:,i*32:(i+1)*32]), axis=1)
        else:
            data['Amplitude stokes ' + e[i]] = np.min(np.abs(y_test[:,i*32:(i+1)*32]), axis=1)

    data = pd.DataFrame(data)
    data['WMAPE'] = (data['stokes I WMAPE'] + data['stokes Q WMAPE'] + data['stokes U WMAPE'] + data['stokes V WMAPE'])/4

    return data

In [ ]:
def make_MSE_WMAPE_error(y_test, y_pred):
    e = ['I', 'Q', 'U', 'V']
    data = {'stokes I MSE': [], 'stokes Q MSE': [], 'stokes U MSE': [], 'stokes V MSE': [], 'stokes I WMAPE': [], 'stokes Q WMAPE': [], 'stokes U WMAPE': [], 'stokes V WMAPE': [], 'WMAPE': [], 'Amplitude stokes I':[], 'Amplitude stokes Q':[], 'Amplitude stokes U':[], 'Amplitude stokes V':[]}
    for i in range(4):
        wmape = weighted_mean_absolute_percentage_error(y_test[:,i*32:(i+1)*32], y_pred[:,i*32:(i+1)*32])
        mse = tf.keras.metrics.mean_squared_error(y_test[:,i*32:(i+1)*32], y_pred[:,i*32:(i+1)*32]).numpy()
        data['stokes ' + e[i] + ' MSE'] = mse
        data['stokes ' + e[i] + ' WMAPE'] = wmape
        if i != 0:
            data['Amplitude stokes ' + e[i]] = np.max(np.abs(y_test[:,i*32:(i+1)*32]), axis=1)
        else:
            data['Amplitude stokes ' + e[i]] = np.min(np.abs(y_test[:,i*32:(i+1)*32]), axis=1)

    data['WMAPE'] = (data['stokes I WMAPE'] + data['stokes Q WMAPE'] + data['stokes U WMAPE'] + data['stokes V WMAPE']) / 4
    return pd.DataFrame(data)

In [ ]:
def make_inversions_csv(x, y, input_scaler, output_scaler, solutions, phases, times, errors, model):
    d = {'fmag': [], 'incl': [], 'alpha': [], 'beta': [], 'gamma': [], 'x2': [], 'x3': [], 
         'fmag pred': [], 'incl pred': [], 'alpha pred': [], 'beta pred': [], 'gamma pred': [], 
         'x2 pred': [], 'x3 pred': [], 'time': [], 'error': []}

    for i in range(phases.shape[1]):
        d['stokes I MSE ph'+str(i)] = []
        d['stokes Q MSE ph'+str(i)] = []
        d['stokes U MSE ph'+str(i)] = []
        d['stokes V MSE ph'+str(i)] = []

        d['stokes I WMAPE ph'+str(i)] = []
        d['stokes Q WMAPE ph'+str(i)] = []
        d['stokes U WMAPE ph'+str(i)] = []
        d['stokes V WMAPE ph'+str(i)] = []
        d['WMAPE ph'+str(i)] = []
    
    final_solutions = input_scaler.inverse_transform(solutions)

    for i in range(len(x)):
        input_data = np.tile(solutions[i][:7], phases.shape[1]).reshape(phases.shape[1], 7)
        input_data = np.column_stack([input_data, phases[i]])
        y_pred = output_scaler.inverse_transform(model.predict(input_data, batch_size=phases.shape[1], verbose=0))
        y_test = y[i].reshape(phases.shape[1], 128)

        tmp = make_MSE_WMAPE_error(y_test, y_pred)
        for ph in range(phases.shape[1]):
            d['stokes I MSE ph'+str(ph)].append(tmp.iloc[ph]['stokes I MSE'])
            d['stokes Q MSE ph'+str(ph)].append(tmp.iloc[ph]['stokes Q MSE'])
            d['stokes U MSE ph'+str(ph)].append(tmp.iloc[ph]['stokes U MSE'])
            d['stokes V MSE ph'+str(ph)].append(tmp.iloc[ph]['stokes V MSE'])

            d['stokes I WMAPE ph'+str(ph)].append(tmp.iloc[ph]['stokes I WMAPE'])
            d['stokes Q WMAPE ph'+str(ph)].append(tmp.iloc[ph]['stokes Q WMAPE'])
            d['stokes U WMAPE ph'+str(ph)].append(tmp.iloc[ph]['stokes U WMAPE'])
            d['stokes V WMAPE ph'+str(ph)].append(tmp.iloc[ph]['stokes V WMAPE'])

            d['WMAPE ph'+str(ph)].append(tmp.iloc[ph]['WMAPE'])

        d['fmag'].append(x[i][0])
        d['incl'].append(x[i][1])
        d['alpha'].append(x[i][2])
        d['beta'].append(x[i][3])
        d['gamma'].append(x[i][4])
        d['x2'].append(x[i][5])
        d['x3'].append(x[i][6])

        d['fmag pred'].append(final_solutions[i][0])
        d['incl pred'].append(final_solutions[i][1])
        d['alpha pred'].append(final_solutions[i][2])
        d['beta pred'].append(final_solutions[i][3])
        d['gamma pred'].append(final_solutions[i][4])
        d['x2 pred'].append(final_solutions[i][5])
        d['x3 pred'].append(final_solutions[i][6])

        d['time'].append(times[i])
        d['error'].append(errors[i])

    rdf = pd.DataFrame(d)
    return rdf

# Load Data

In [ ]:
data_path = '../data/'
results_path = 'new results/'

size_dataset = int(3.5e6)

In [ ]:
df = pd.read_csv('../fe6311/cossam_train_data_high.csv', nrows=size_dataset)
print(df.shape)
df.head()

In [ ]:
test_df = pd.read_csv('../data/cossam_data_test_inversions_6311_high.csv')
print(test_df.shape)
test_df.head()

In [ ]:
all_stk_dim = int((df.shape[1] - 11)/5)
resolution = 32
mid_point = int(all_stk_dim/2)
start = int(mid_point - np.floor(resolution/2))
end = int(mid_point + np.ceil(resolution/2))

cols = ['fmag', 'incl', 'alpha', 'beta', 'gamma', 'y2', 'y3', 'phase']
stks = [] 
y_cols = ['stki_', 'stkq_', 'stku_','stkv_']
for s in y_cols:
    for i in range(start, end):
        stks.append(s+str(i))
stk_dim = int(len(stks) / 4)
cols = ['fmag', 'incl', 'alpha', 'beta', 'gamma', 'y2', 'y3']
x = df[cols + ['phase']]
y = df[stks]


phases = 7
stks = []
#
y_cols = ['stki_ph', 'stkq_ph', 'stku_ph', 'stkv_ph']
for ph in range(phases):
    ycs = [e+str(ph) for e in y_cols]
    for s in ycs:
        for i in range(start, end):
            stks.append(s+'_'+str(i-1))

xt = test_df[cols + ['phase_ph'+str(i) for i in range(phases)]]
yt = test_df[stks]

# Preprocessing

In [ ]:
x_train = x.to_numpy()
x_test = xt.to_numpy()
y_train = y.to_numpy()
y_test = yt.to_numpy()

In [ ]:
scalerX = StandardScaler()
x_train_s = scalerX.fit_transform(x_train[:, :-1])
phase_scaler = StandardScaler()
phase_s = phase_scaler.fit_transform(x_train[:, -1].reshape(len(x_train), 1))
x_train_s = np.concatenate((x_train_s, phase_s), axis=1)

x_test_s = scalerX.transform(x_test[:, :7])
for i in range(phases):
    phase_s = phase_scaler.transform(x_test[:, 7+i:8+i])
    x_test_s = np.concatenate((x_test_s, phase_s), axis=1)

scalerY = MaxAbsScaler()
y_train_s = scalerY.fit_transform(y_train)

stk_dim = resolution*4
tmp = []
for i in range(phases):
    tmp.append(scalerY.transform(y_test[:,i*stk_dim:(i+1)*stk_dim]))
y_test_s = np.concatenate(tmp, axis=1)

In [ ]:
model = tf.keras.models.load_model('new results/1M_standard_maxabs_verylow.h5')

# Recover Magnetic Field

In [ ]:
k = 2048
phases = 7
n = 100
all_phases = []
for j in range(10):
    times = []
    errors = []
    solutions = []
    print('------------------------')
    print('Start', j, 'run')
    print('------------------------')
    for i in range(n):
        s = time.time()
        tmp = np.copy(y_test_s[i])
        pso = PSO(n_particles=k, iters=50, bounds=(x_train_s.min(), x_train_s.max()), dims=7, 
              y_true=np.tile(tmp, k).reshape(k, stk_dim*phases), phases=list(x_test_s[i][-phases:]), 
              mode='random', C1=1, C2=1, weights=[1,1,1,1])
        pso.fit()
        e = time.time()
        times.append(e - s)
        errors.append(pso.gbest_fit)
        solutions.append(pso.gbest_pos)
        all_phases.append(pso.phases)
        if i % 25 == 0:
            print(i, 'iterations')
    print(n, 'iterations')
        
        

    times = np.array(times)
    errors = np.array(errors)
    solutions = np.array(solutions)
    rdf = make_inversions_csv(x_test[:n], y_test[:n], scalerX, scalerY, solutions, np.array(all_phases), times, errors, model)
    rdf.to_csv('new results/pso_inversions_2048p_50iters_'+str(phases)+'phases_'+str(j)+'_high.csv', index=False)
    rdf.head()